In [42]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import random

options = webdriver.ChromeOptions()
options.binary_location = r"/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
options.add_argument(r'/Users/guanmengting/Library/Application Support/Google/Chrome/')
# options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
# options.add_argument(r"C:\Users\yufeng\AppData\Local\Google\Chrome\User Data\Profile 2")
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

In [75]:
def current_window():
    windowstabs = driver.window_handles
    current = driver.current_window_handle
    return windowstabs.index(current)

def switch_window(index):
    windowstabs = driver.window_handles
    driver.switch_to.window(windowstabs[index])
    return current_window()

def close_window():
    driver.close()
    switch_window(-1)
    
def parse_web_page():
    switch_window(-1)
    if driver.find_elements(By.CSS_SELECTOR, 'div.J_MIDDLEWARE_FRAME_WIDGET'):
        input("完成验证后回车")
    item_name = driver.find_element(By.CSS_SELECTOR, 'h1.ItemHeader--mainTitle--3CIjqW5').text if driver.find_element(By.CSS_SELECTOR, 'h1.ItemHeader--mainTitle--3CIjqW5') else ""
    sale_month = driver.find_element(By.CSS_SELECTOR, 'span.ItemHeader--salesDesc--srlk2Hv').text if driver.find_element(By.CSS_SELECTOR, 'span.ItemHeader--salesDesc--srlk2Hv') else ""
    skuValueName = [element.text for element in driver.find_elements(By.CSS_SELECTOR, 'span.skuValueName')] if driver.find_elements(By.CSS_SELECTOR, 'span.skuValueName') else []
    price = driver.find_element(By.CSS_SELECTOR, 'span.Price--priceText--2nLbVda').text if driver.find_element(By.CSS_SELECTOR, 'span.Price--priceText--2nLbVda') else ""
    sales_price = driver.find_elements(By.CSS_SELECTOR, 'span.Price--extraPriceText--2dbLkGw')
    sales_price = sales_price[0].text if sales_price else -1

    ret = {
        "item_name": item_name,
        "sale_month": re.search(r'\d+', sale_month).group(0),
        "skuValueName": skuValueName,
        "price": price,
        "sales_price": sales_price,
        "link": driver.current_url
    }
    close_window()
    return ret

def sleep():
    time.sleep(2 * random.random() + 4)
    

In [52]:
# Step1: 扫码登录淘宝
# Step2: 然后进到淘宝店里面
driver.get("https://www.taobao.com")
try:
    driver.find_element(By.LINK_TEXT, "亲，请登录").click()
    driver.find_element(By.CSS_SELECTOR, "i.iconfont.icon-qrcode").click()
except Exception as e:
    pass

[<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.42A238F8962978D845C8729E38DF0444.d.DF542295F51E6C467A274E0196BE59AA.e.65")>]

In [76]:
# Step3: 关闭其他标签页，运行完这段代码，所有宝贝页面要在active状态
switch_window(-1)

1

In [77]:
result = []
Pagination = True
page = 1

while Pagination:
    index = 1
    itemList = driver.find_elements(By.CSS_SELECTOR, "div.J_TItems > *") + driver.find_elements(By.CSS_SELECTOR, "div.shop-hesper-bd > *")
    for item in itemList:
        try:
            if item.get_attribute("class") in ["item3line1", "item4line1", "item5line1"]:
                for spu in item.find_elements(By.CSS_SELECTOR, "dd.detail"):
                    spu.find_element(By.CSS_SELECTOR, 'a.item-name').click()
                    print(f"正在获取第{page}页第{index}个产品", end='\r')
                    sleep()
                    resultEntity = parse_web_page()
                    total_sale = spu.find_element(By.CSS_SELECTOR, 'span.sale-num').text
                    resultEntity.update({'total_sale': re.search(r'\d+', total_sale.replace('万', '0000')).group(0)})
                    result.append(resultEntity)
                    index += 1
    
            if item.get_attribute("class") == "pagination":
                nextPage = item.find_element(By.LINK_TEXT, "下一页")
                if nextPage.get_attribute("class") == "disable":
                    Pagination = False
                else:
                    item.find_element(By.LINK_TEXT, "下一页").click()
                    page += 1
                    sleep()
                break
        except Exception as e:
            print(e)
            close_window()
            

<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.FBCC808106C0C8BA09653029FD5699CE.d.AC6836672633484FAEAD529DF7B92D1F.e.639")>
<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.FBCC808106C0C8BA09653029FD5699CE.d.AC6836672633484FAEAD529DF7B92D1F.e.636")>
<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.FBCC808106C0C8BA09653029FD5699CE.d.AC6836672633484FAEAD529DF7B92D1F.e.697")>
<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.FBCC808106C0C8BA09653029FD5699CE.d.AC6836672633484FAEAD529DF7B92D1F.e.698")>
<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42ba", element="f.FBCC808106C0C8BA09653029FD5699CE.d.AC6836672633484FAEAD529DF7B92D1F.e.699")>
<selenium.webdriver.remote.webelement.WebElement (session="7f8b96e53e4931054e20584a44ca42b

KeyboardInterrupt: 

In [75]:
df = pd.DataFrame(result)
df

,item_name,sale_month,skuValueName,price,sales_price,link,total_sale
0,高档精致金属书签定制礼盒文创产品小礼品故宫风敦煌创意古典中国风博物馆古风生日礼物清华大学纪念...,2000,"[乐舞飞天 - 单枚礼盒装, 九色迷鹿 - 单枚礼盒装, 宝相佛莲 - 单枚礼盒装, 琵琶弦...",18.2,-1,https://detail.tmall.com/item.htm?abbucket=12&...,40000
1,仙鹤折扇玫瑰金金属艺术流苏书签礼盒 创意古典中国风送老师学生精美文艺文创产品纪念品商务小礼品...,300,"[九色灵狐 折扇书签, 云鹤浪空 折扇书签, 凤舞九天 折扇书签, 敦煌飞天 折扇书签, 江...",26.8,24.8,https://detail.tmall.com/item.htm?abbucket=12&...,3000
2,千里江山金属黄铜书签签字笔笔记本U盘套装企业定制logo刻字精美古典中国风创意文创产品纪念商...,3000,"[千里江山文艺两件套（酸枝）, 千里江山文艺两件套（黑檀）, 千里江山文艺两件套（紫檀）, ...",29.4,27.4,https://detail.tmall.com/item.htm?abbucket=12&...,8000
3,复古典红木质流苏书签 中国风创意文艺黑檀木学生免费刻字送老师同学个性手工公司商务小礼品个性免费定制,600,"[花梨木/天道酬勤, 花梨木/上善若水, 花梨木/静思, 花梨木/远见, 花梨木/梅, 花梨...",18.8,-1,https://detail.tmall.com/item.htm?abbucket=12&...,5000
4,故宫金属千里江山图黄铜书签刻字 古典中国风送老师学生用简约文艺创意文创产品精美礼物纪念品礼品...,100,"[千里江山图单枚书签, 池塘睡莲图单枚书签, 宋微宗瑞鹤图单枚书签, 神奈川冲浪里单枚书签,...",22.8,-1,https://detail.tmall.com/item.htm?abbucket=12&...,3000


In [21]:
df.to_excel("result.xlsx")